In [1]:
import pandas as pd

In [26]:
#oct = pd.read_csv('2019-Oct.csv')
#nov = pd.read_csv('2019-Nov.csv')
#dec = pd.read_csv('2019-Dec.csv')
#jan = pd.read_csv('2020-Jan.csv')
#feb = pd.read_csv('2020-Feb.csv')
#merge data
frames = [oct, nov, dec, jan, feb]
df = pd.concat(frames)

In [30]:
#df = df[1500:2500]

#df.dtypes
#df['user_session']=df['user_session'].astype('|S')
df.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
4156677,2020-02-29 23:59:32 UTC,view,5885416,1487580005092295511,NaN,grattol,6.27,622082947,fb29909b-6ef5-4662-b4ee-288e73e5dc10
4156678,2020-02-29 23:59:39 UTC,cart,5550686,1487580008145748965,NaN,NaN,1.11,459705611,05d2add3-01f7-47ee-8364-27341673227f
4156679,2020-02-29 23:59:45 UTC,view,5850628,1602943681873052386,NaN,grattol,5.24,622090043,ab7d349f-db5d-4790-8ab1-31e5c894459d
4156680,2020-02-29 23:59:54 UTC,view,5716351,1487580010872045658,NaN,irisk,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d
4156681,2020-02-29 23:59:59 UTC,view,5927709,1487580013950664926,NaN,de.lux,3.17,621669991,daa3c677-5360-4327-847e-d29658716b99


In [32]:
final = pd.read_csv('final.csv')

In [29]:
df['user_session']=df['user_session'].apply(lambda x : [int(i,16) for i in x.split('-')])
df.tail()

AttributeError: 'float' object has no attribute 'split'

In [28]:
#add attribute event time
df['event_time'] = pd.to_datetime(df['event_time'])
df['event_hour'] = pd.Series(df.event_time).dt.hour
df.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_hour
2495,2019-10-01 02:08:29+00:00,remove_from_cart,5812052,1487580005427839846,NaN,irisk,2.48,550154276,c937f69d-67c9-4835-b854-fba481ddd7c8,2
2496,2019-10-01 02:08:29+00:00,remove_from_cart,5812052,1487580005427839846,NaN,irisk,2.48,550154276,c937f69d-67c9-4835-b854-fba481ddd7c8,2
2497,2019-10-01 02:08:33+00:00,cart,5736463,1487580004857414477,NaN,lianail,5.54,555460377,225088c5-1414-41fd-91a4-dc8f91350729,2
2498,2019-10-01 02:08:41+00:00,view,5652894,1487580008699397117,NaN,sophin,20.63,555455025,320f6021-30ac-4a58-ae17-bac1cc32aac3,2
2499,2019-10-01 02:08:44+00:00,remove_from_cart,5775528,1487580008187692007,NaN,NaN,4.44,550154276,c937f69d-67c9-4835-b854-fba481ddd7c8,2


In [29]:
df1 = df[['user_id','event_hour']]
df1['count'] = 1
df1 = df1.groupby(['user_id','event_hour']).sum()
df1.sort_values(by=['count'],ascending=False, inplace=True)
df1 = df1.reset_index(level = 'event_hour')
df1 = df1.groupby('user_id').first()
final = df[['user_id','event_hour']]
final.tail(20)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,event_hour
2480,555415545,2
2481,341647197,2
2482,555415545,2
2483,541654163,2
2484,555415545,2
2485,549209944,2
2486,550154276,2
2487,552186056,2
2488,550154276,2
2489,550154276,2


In [30]:
#add attribute total_purchase
df2 = df[df['event_type'] == 'purchase']
df2 = df2[['user_id','price']].groupby('user_id').sum()
df2.rename(columns = {'price':'total_purchase'}, inplace = True) 
final = final.join(df2,on='user_id')
final.head()

,user_id,event_hour,total_purchase
1500,464191317,1,206.02
1501,555451436,1,NaN
1502,555170065,1,NaN
1503,555170065,1,NaN
1504,464037166,1,NaN


In [31]:
# add attribute session count
df3 = df[['user_id','user_session']]
df3 = df3.groupby(['user_id','user_session']).first()
df3['session_count'] = 1
df3 = df3.groupby('user_id').sum()

NameError: name 'final' is not defined

In [33]:
final = final.join(df3,on='user_id')
final.head()

,user_id,event_hour,total_purchase,purchase_times,cart_times,view_times,session_count
0,465496,16,NaN,NaN,NaN,3.0,2.0
1,1120748,15,NaN,NaN,NaN,1.0,1.0
2,1180452,14,NaN,NaN,NaN,1.0,1.0
3,1458813,12,NaN,NaN,NaN,31.0,4.0
4,2038666,13,NaN,NaN,NaN,1.0,1.0


In [129]:
df4 = df[df['event_type'] == 'purchase']
df4 = df4[['user_id','event_type']].groupby('user_id').count()
df4.rename(columns = {'event_type':'purchase_times'}, inplace = True) 
final = final.join(df4,on='user_id')
final.head()

,event_hour,total_purchase,session_count,purchase_times
user_id,,,,
50764735,1.0,NaN,1,NaN
223048543,1.0,NaN,1,NaN
265372231,1.0,NaN,1,NaN
278482846,1.0,NaN,1,NaN
279141909,1.0,NaN,4,NaN


In [130]:
df5 = df[df['event_type'] == 'cart']
df5 = df5[['user_id','event_type']].groupby('user_id').count()
df5.rename(columns = {'event_type':'cart_times'}, inplace = True) 
df6 = df[df['event_type'] == 'view']
df6 = df6[['user_id','event_type']].groupby('user_id').count()
df6.rename(columns = {'event_type':'view_times'}, inplace = True) 
final = final.join(df5,on='user_id')
final = final.join(df6,on='user_id')
final.head()

,event_hour,total_purchase,session_count,purchase_times,cart_times,view_times
user_id,,,,,,
50764735,1.0,NaN,1,NaN,NaN,1.0
223048543,1.0,NaN,1,NaN,NaN,1.0
265372231,1.0,NaN,1,NaN,4.0,2.0
278482846,1.0,NaN,1,NaN,NaN,2.0
279141909,1.0,NaN,4,NaN,NaN,4.0


In [34]:
final = final.fillna(0)
final.head()

,user_id,event_hour,total_purchase,purchase_times,cart_times,view_times,session_count
0,465496,16,0.0,0.0,0.0,3.0,2.0
1,1120748,15,0.0,0.0,0.0,1.0,1.0
2,1180452,14,0.0,0.0,0.0,1.0,1.0
3,1458813,12,0.0,0.0,0.0,31.0,4.0
4,2038666,13,0.0,0.0,0.0,1.0,1.0


In [35]:
# add attributes
final['purchase_per_session'] = final.total_purchase/final.session_count
final['purchase_times_per_session'] = final.purchase_times/final.session_count
final['cart_per_view'] = final.cart_times/final.view_times
final.head()

,user_id,event_hour,total_purchase,purchase_times,cart_times,view_times,session_count,purchase_per_session,purchase_times_per_session,cart_per_view
0,465496,16,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0
1,1120748,15,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1180452,14,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1458813,12,0.0,0.0,0.0,31.0,4.0,0.0,0.0,0.0
4,2038666,13,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [1]:
#drop support columns
#out = final.drop(['cart_times','view_times','purchase_times'],axis=1)
#out.tail()
out['purchase_per_session'].mean()

NameError: name 'final' is not defined

In [38]:
out.to_csv('user profile.csv')